# Usage and bug
Directly run this pipeline with `./Multi_tissues.ipynb`.
However, you will see an error `./Multi_tissues.ipynb: line 133: syntax error: unexpected end of file`

In [1]:
[Z]
R:
    filenames <- list.files(path = "/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_covariates", pattern = "*.txt", full.names = TRUE)
    Z <- lapply(filenames, function(x) t(as.matrix(read.table(file = x, header = TRUE, sep = '\t', quote = "", row.names = 1))))

In [ ]:
[y]
bash:
    zless /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz | grep ENSG00000140265 > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ENSG00000140265.txt 
R:
    y_raw <- scan(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ENSG00000140265.txt", what = "", sep = "\n")
    y <- strsplit(x = y_raw, split = "[[:space:]]+")
    y <- lapply(y, function(x) x[-1:-4])
    y <- lapply(y, as.numeric)

In [ ]:
[X]
bash:
    awk '$4 ~ /ENSG00000140265/ {print $0}' /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/gene_TSS.csv | head -n 1 | awk '{$3=$2+1000000} {$2=$2-1000000} {print $1"\t"$2"\t"$3"\t"$4}' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ENSG00000140265.bed
    tabix /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz chr15:42358172-44358172 | sed -e 's/0|0/0/g' -e 's/0|1/1/g' -e 's/1|0/1/g' -e 's/1|1/2/g' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ENSG00000140265_genotype.txt
    
R:
    X <- read.csv(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/ENSG00000140265_genotype.txt", sep = '\t', header = FALSE, row.names = 3, stringsAsFactors = FALSE)
    X <- X[,-(1:8)]
    samplenames_X <- scan(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/samplenames_X.txt", what = character(), quote = "")
    samplenames_X <- samplenames_X[-(1:9)]
    colnames(X) <- samplenames_X
    X <- as.matrix(x = X)
    X <- t(X)

In [ ]:
[y_res]
R:
    y_res <- lapply(1:49, function(i) .lm.fit(x = Z[[i]], y = y[[i]])$residuals)

In [ ]:
[save]
R:
    saveRDS(object = list(X = X, y = y, Z = Z, y_res = y_res), file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/Multi_Tissues.ENSG00000140265.RDS")